In [1]:
from pybaseball import statcast
from pybaseball import playerid_lookup
from pybaseball import statcast_pitcher
from pybaseball import pitching_stats
from pybaseball import batting_stats
from pybaseball import playerid_reverse_lookup
import pandas as pd
import numpy as np
import pyodbc
from pybaseball import cache
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import psycopg2
from sqlalchemy import create_engine
import pyodbc
from dateutil.relativedelta import relativedelta
from datetime import date

#### Connect to SQL Database

In [2]:
param_dict = {
    "host": "localhost",
    "database": "Baseball",
    "user": "postgres",
    "password": "Bullets10!"
}

param_dict = {
    "host": "localhost",
    "database": "Baseball",
    "user": "postgres",
    "password": "Bullets10!"
}

def connect(params_dict):
    "Connecting to PostgreSQL Database Server"
    conn = None
    try:
        print("Connecting to the PostgreSQL Database")
        conn = psycopg2.connect(**params_dict)
    except (Exception, psycopg2.DataError) as error:
        print(error)
    return conn

connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
    param_dict['user'],
    param_dict['password'],
    param_dict['host'],
    param_dict['database']
)
engine = create_engine(connect)

#### Load All Final Pitch Data

Final Pitch = Pitch that resulted in an event (In Play / Strikeout)

In [ ]:
data = pd.read_sql_query("select * from final_pitch", con=engine)
data.head()